<a href="https://colab.research.google.com/github/AnatolyKayda/hackathon-sem-3/blob/master/LoRA_kayda_hackaton3sem_rccmsu_ruadapt_llama2_7b_v0_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Очистка и подготовка данных

In [1]:
! pip install datasets --progress-bar off

In [2]:
from datasets import load_dataset
import pandas as pd

In [3]:
dataset = load_dataset("d0rj/geo-reviews-dataset-2023")
work_data = pd.DataFrame(dataset['train'])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
work_data.head()

,address,name_ru,rating,rubrics,text
0,"Екатеринбург, ул. Московская / ул. Волгоградск...",Московский квартал,3,Жилой комплекс,Московский квартал 2.\nШумно : летом по ночам ...
1,"Московская область, Электросталь, проспект Лен...",Продукты Ермолино,5,Магазин продуктов;Продукты глубокой заморозки;...,"Замечательная сеть магазинов в общем, хороший ..."
2,"Краснодар, Прикубанский внутригородской округ,...",LimeFit,1,Фитнес-клуб,"Не знаю смутят ли кого-то данные правила, но я..."
3,"Санкт-Петербург, проспект Энгельса, 111, корп. 1",Snow-Express,4,Пункт проката;Прокат велосипедов;Сапсёрфинг,Хорошие условия аренды. \nДружелюбный персонал...
4,"Тверь, Волоколамский проспект, 39",Студия Beauty Brow,5,"Салон красоты;Визажисты, стилисты;Салон бровей...",Топ мастер Ангелина топ во всех смыслах ) Немн...


In [5]:
work_data = work_data.dropna(subset=['text', 'name_ru', 'rating'])
work_data = work_data.drop_duplicates(subset=['text']).reset_index(drop=True)
work_data['text'] = work_data['text'].str.replace('\\n', ' ')
work_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 498918 entries, 0 to 498917
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   address  498918 non-null  object
 1   name_ru  498918 non-null  object
 2   rating   498918 non-null  int64 
 3   rubrics  498918 non-null  object
 4   text     498918 non-null  object
dtypes: int64(1), object(4)
memory usage: 19.0+ MB


In [6]:
work_data.head()

,address,name_ru,rating,rubrics,text
0,"Екатеринбург, ул. Московская / ул. Волгоградск...",Московский квартал,3,Жилой комплекс,Московский квартал 2. Шумно : летом по ночам д...
1,"Московская область, Электросталь, проспект Лен...",Продукты Ермолино,5,Магазин продуктов;Продукты глубокой заморозки;...,"Замечательная сеть магазинов в общем, хороший ..."
2,"Краснодар, Прикубанский внутригородской округ,...",LimeFit,1,Фитнес-клуб,"Не знаю смутят ли кого-то данные правила, но я..."
3,"Санкт-Петербург, проспект Энгельса, 111, корп. 1",Snow-Express,4,Пункт проката;Прокат велосипедов;Сапсёрфинг,Хорошие условия аренды. Дружелюбный персонал....
4,"Тверь, Волоколамский проспект, 39",Студия Beauty Brow,5,"Салон красоты;Визажисты, стилисты;Салон бровей...",Топ мастер Ангелина топ во всех смыслах ) Немн...


In [48]:
import re
def clean_review_text(text: str) -> str:
    text = text.lower()  # Приводим к нижнему регистру
    text = re.sub(r"<[^>]+>", "", text)  # Удаляем HTML-теги
    text = re.sub(r"[^\w\s,.!?()]+", "", text)  # Удаляем спецсимволы, кроме пунктуации и скобок
    text = re.sub(r"\)\)+", " ", text)  # Заменяем смайлики вида "))" на пробел
    text = re.sub(r"\s+", " ", text).strip()  # Убираем лишние пробелы
    text = re.sub(r"[\n\r]+", " ", text)  # Заменяем переносы строк на пробелы
    return text


def clean_rubrics(rubrics_: str) -> str:
    return rubrics_.lower().strip()


def clean_name_ru(name_ru: str) -> str:
    return name_ru.lower().strip()


def clean_address(address: str) -> str:
    address = address.strip()
    return re.sub(r"\s+", " ", address)


work_data['text'] = work_data['text'].apply(clean_review_text)
work_data['rubrics'] = work_data['rubrics'].apply(clean_rubrics)
work_data['name_ru'] = work_data['name_ru'].apply(clean_name_ru)
work_data['address'] = work_data['address'].apply(clean_address)

# Преобразование рейтинга в числовой формат
work_data['rating'] = work_data['rating'].astype(float)

# Удаление строк с пропущенными значениями
work_data.dropna(subset=['address', 'name_ru', 'rubrics', 'rating', 'text'], inplace=True)
work_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 498912 entries, 0 to 498917
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   address    498912 non-null  object 
 1   name_ru    498912 non-null  object 
 2   rating     498912 non-null  float64
 3   rubrics    498912 non-null  object 
 4   text       498912 non-null  object 
 5   key_words  498912 non-null  object 
dtypes: float64(1), object(5)
memory usage: 26.6+ MB


In [8]:
work_data.head()

,address,name_ru,rating,rubrics,text
0,"Екатеринбург, ул. Московская / ул. Волгоградск...",московский квартал,3.0,жилой комплекс,московский квартал 2. шумно летом по ночам дик...
1,"Московская область, Электросталь, проспект Лен...",продукты ермолино,5.0,магазин продуктов;продукты глубокой заморозки;...,"замечательная сеть магазинов в общем, хороший ..."
2,"Краснодар, Прикубанский внутригородской округ,...",limefit,1.0,фитнес-клуб,"не знаю смутят ли когото данные правила, но я ..."
3,"Санкт-Петербург, проспект Энгельса, 111, корп. 1",snow-express,4.0,пункт проката;прокат велосипедов;сапсёрфинг,хорошие условия аренды. дружелюбный персонал. ...
4,"Тверь, Волоколамский проспект, 39",студия beauty brow,5.0,"салон красоты;визажисты, стилисты;салон бровей...",топ мастер ангелина топ во всех смыслах ) немн...


In [9]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords

# Убедитесь, что у вас загружены стоп-слова
nltk.download('stopwords', quiet=True)
russian_stopwords = stopwords.words('russian')

# Функция для извлечения трех ключевых слов
def extract_keywords(text):
    text = text.strip()
    if not text:  # Проверка на пустую строку
        return ''

    vectorizer = CountVectorizer(stop_words=russian_stopwords, max_features=3)

    try:
        X = vectorizer.fit_transform([text])
        # Извлекаем ключевые слова
        keywords = vectorizer.get_feature_names_out()
        return ', '.join(keywords)
    except:
        return ''

In [10]:
work_data['key_words'] = work_data['text'].apply(extract_keywords)

In [11]:
work_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 498918 entries, 0 to 498917
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   address    498918 non-null  object 
 1   name_ru    498918 non-null  object 
 2   rating     498918 non-null  float64
 3   rubrics    498918 non-null  object 
 4   text       498918 non-null  object 
 5   key_words  498918 non-null  object 
dtypes: float64(1), object(5)
memory usage: 22.8+ MB


In [12]:
weird_data = work_data[work_data['key_words']=='']
weird_data.info()
#проще удалить эти отзывы!

<class 'pandas.core.frame.DataFrame'>
Index: 6 entries, 53878 to 489715
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   address    6 non-null      object 
 1   name_ru    6 non-null      object 
 2   rating     6 non-null      float64
 3   rubrics    6 non-null      object 
 4   text       6 non-null      object 
 5   key_words  6 non-null      object 
dtypes: float64(1), object(5)
memory usage: 336.0+ bytes


In [13]:
work_data = work_data[work_data['key_words']!='']
work_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 498912 entries, 0 to 498917
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   address    498912 non-null  object 
 1   name_ru    498912 non-null  object 
 2   rating     498912 non-null  float64
 3   rubrics    498912 non-null  object 
 4   text       498912 non-null  object 
 5   key_words  498912 non-null  object 
dtypes: float64(1), object(5)
memory usage: 26.6+ MB


In [14]:
work_data.to_csv('work_data.csv', index=False)

print("Данные успешно очищены и сохранены в 'work_data.csv'.")

Данные успешно очищены и сохранены в 'work_data.csv'.


#LoRA

In [15]:
!pip install -q accelerate --progress-bar off
!pip install -q peft --progress-bar off
!pip install -q bitsandbytes --progress-bar off
!pip install -q transformers --progress-bar off
!pip install -q trl --progress-bar off

In [16]:
import os
from random import randrange
from functools import partial
import torch
from datasets import load_dataset
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer,
                          BitsAndBytesConfig,
                          HfArgumentParser,
                          Trainer,
                          TrainingArguments,
                          DataCollatorForLanguageModeling,
                          EarlyStoppingCallback,
                          pipeline,
                          logging,
                          set_seed)

import bitsandbytes as bnb
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftModel, AutoPeftModelForCausalLM
from trl import SFTTrainer

In [17]:
def create_bnb_config(load_in_4bit, bnb_4bit_use_double_quant, bnb_4bit_quant_type, bnb_4bit_compute_dtype):
    """
    Configures model quantization method using bitsandbytes to speed up training and inference

    :param load_in_4bit: Load model in 4-bit precision mode
    :param bnb_4bit_use_double_quant: Nested quantization for 4-bit model
    :param bnb_4bit_quant_type: Quantization data type for 4-bit model
    :param bnb_4bit_compute_dtype: Computation data type for 4-bit model
    """

    bnb_config = BitsAndBytesConfig(
        load_in_4bit = load_in_4bit,
        bnb_4bit_use_double_quant = bnb_4bit_use_double_quant,
        bnb_4bit_quant_type = bnb_4bit_quant_type,
        bnb_4bit_compute_dtype = bnb_4bit_compute_dtype,
    )

    return bnb_config

In [18]:
import time
def load_model(model_name, bnb_config):
    """
    Loads model and model tokenizer

    :param model_name: Hugging Face model name
    :param bnb_config: Bitsandbytes configuration
    """

    # Get number of GPU device and set maximum memory
    n_gpus = torch.cuda.device_count()
    max_memory = f'{40960}MB'
    time.sleep(1)
    # Load model
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config = bnb_config,
        device_map = "auto", # dispatch the model efficiently on the available resources
        max_memory = {i: max_memory for i in range(n_gpus)},
    )

    time.sleep(1)
    # Load model tokenizer with the user authentication token
    tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token = False)

    # Set padding token as EOS token
    tokenizer.pad_token = tokenizer.eos_token

    return model, tokenizer

In [19]:


################################################################################
# transformers parameters
################################################################################

# The pre-trained model from the Hugging Face Hub to load and fine-tune

#model_name = "sberbank-ai/rugpt3small_based_on_gpt2"
#model_name = "rccmsu/ruadapt_llama2_7b_v0.1"



################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
load_in_4bit = True

# Activate nested quantization for 4-bit base models (double quantization)
bnb_4bit_use_double_quant = True

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Compute data type for 4-bit base models
bnb_4bit_compute_dtype = torch.float32

In [20]:
# Load model from Hugging Face Hub with model name and bitsandbytes configuration

bnb_config = create_bnb_config(load_in_4bit, bnb_4bit_use_double_quant, bnb_4bit_quant_type, bnb_4bit_compute_dtype)

In [21]:
from huggingface_hub import notebook_login
notebook_login()

In [22]:
model, tokenizer = load_model(model_name, bnb_config)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:810: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


### Загрузка датасета

Будем использовать заранее подготовленный датасет с инструкциями.

В данном случае это csv-таблица с 3 колонками:

* Инструкция
* Текст
* Класс

Мы будем использовать стандартный генератор датасета типа `csv` (потому что у нас файл CSV). Аналогично здесь мог бы быть файл JSON и типа датасет `json`. По умолчанию все записи относятся к разделению `train`, который мы получим с помощью параметра `split`.

In [23]:
dataset = load_dataset("csv", data_files = '/content/work_data.csv', split = "train")

Generating train split: 0 examples [00:00, ? examples/s]

In [24]:
print(f'Number of prompts: {len(dataset)}')
print(f'Column names are: {dataset.column_names}')

Number of prompts: 498912
Column names are: ['address', 'name_ru', 'rating', 'rubrics', 'text', 'key_words']


Функция `load_dataset` преобразует файл CSV в словарь промтов. Мы можем просмотреть объекты, используя случайный индекс.

In [25]:
dataset[randrange(len(dataset))]

{'address': 'Санкт-Петербург, Петергоф, Эрлеровский бульвар, 12/2',
 'name_ru': 'магнит',
 'rating': 5.0,
 'rubrics': 'магазин продуктов;магазин хозтоваров и бытовой химии;супермаркет',
 'text': 'хороший магазин. работает круглосуточно. широкий ассортимент товаров. из минусов отсутствие прайсчекера. ну и большое количество покупателей, хотя, учитывая месторасположение супермаркета, это и понятно. ценники на товары в основном правильные, хотя и бывают исключения, но вопрос решается в пользу покупателя.',
 'key_words': 'ассортимент, бывают, хотя'}

### Создание шаблона промта

Определим функцию `create_prompt_formats` для создания промта для модели на основе содержания промтов в датасте.

In [26]:
def create_prompt_formats(sample):

    # Combine a prompt with the static strings
    categories = sample['rubrics'].split(';')  # Предполагается, что рубрики разделены запятыми
    categories = [cat.strip() for cat in categories]
    categories_str = ', '.join(categories)
    categories = f"Категории:\n{categories_str}"

    rating = f"Рейтинг:\n{sample['rating']}"
    keywords = f"Ключевые слова:\n{sample['key_words']}"
    rewiev = f"Отзыв:\n{sample['text']}"

    # Create a list of prompt template elements
    parts = [part for part in [categories, rating, keywords, rewiev] if part]

    # Join prompt template elements into a single string to create the prompt template
    formatted_prompt = "\n\n".join(parts)

    # Store the formatted prompt template in a new key "text"
    sample["prompt"] = formatted_prompt

    return sample

In [27]:
create_prompt_formats(dataset[randrange(len(dataset))])

{'address': 'Москва, поселение Московский, Киевское шоссе, 23-й километр, 1',
 'name_ru': 'саларис',
 'rating': 5.0,
 'rubrics': 'торговый центр;развлекательный центр',
 'text': 'были в саларис, уезжали в аэропорт, т к. там единственная станция метро для нас была на пути в аэропорт и домой. грандиозный внушительный центр! скоротали отлично время, пообедали.',
 'key_words': 'аэропорт, внушительный, время',
 'prompt': 'Категории:\nторговый центр, развлекательный центр\n\nРейтинг:\n5.0\n\nКлючевые слова:\nаэропорт, внушительный, время\n\nОтзыв:\nбыли в саларис, уезжали в аэропорт, т к. там единственная станция метро для нас была на пути в аэропорт и домой. грандиозный внушительный центр! скоротали отлично время, пообедали.'}

In [28]:
type(dataset)

datasets.arrow_dataset.Dataset

In [29]:
dataset.column_names

['address', 'name_ru', 'rating', 'rubrics', 'text', 'key_words']

### Получение максимальной длины последовательности предобученной модели

Определим функцию `get_max_length` для определения максимальной длины посделовательности для Llama-2-7B model.

Эта функция извлечет конфигурацию модели и попытается найти максимальную длину последовательности из одного из нескольких ключей конфигурации, которые могут ее содержать. Если максимальная длина последовательности не найдена, по умолчанию она будет равна 1024. Мы будем использовать максимальную длину последовательности во время предварительной обработки датасета, чтобы удалить записи, которые превышают эту длину контекста, поскольку предварительно обученная модель не примет их.

In [30]:
def get_max_length(model):
    """
    Extracts maximum token length from the model configuration

    :param model: Hugging Face model
    """

    # Pull model configuration
    conf = model.config
    # Initialize a "max_length" variable to store maximum sequence length as null
    max_length = None
    # Find maximum sequence length in the model configuration and save it in "max_length" if found
    for length_setting in ["n_positions", "max_position_embeddings", "seq_length"]:
        max_length = getattr(model.config, length_setting, None)
        if max_length:
            print(f"Found max lenth: {max_length}")
            break
    # Set "max_length" to 1024 (default value) if maximum sequence length is not found in the model configuration
    if not max_length:
        max_length = 1024
        print(f"Using default max length: {max_length}")
    return max_length

### Токенизирование батчей

Функция `preprocess_batch` будет токенизировать входящий (`batch`) используя  `tokenizer`. Мы устанавливаем параметр максимальной длины последовательности  `max_length`, от которго будет зависеть паддинг либо обрезка данных.

Параметр `truncation = True` приведёт к обрезанию длинных текстов.

Параметр `padding = max_length` будет производить паддинг до максимальной длины.

In [31]:
def preprocess_batch(batch, tokenizer, max_length):
    """
    Tokenizes dataset batch

    :param batch: Dataset batch
    :param tokenizer: Model tokenizer
    :param max_length: Maximum number of tokens to emit from the tokenizer
    """

    return tokenizer(
        batch["prompt"],
        max_length = max_length,
        truncation = True,
    )

### Предобработка датасета

1. Создание промтов через функцию `create_prompt_formats`.
2. Токенизирование батчей через функцию `preprocess_batch`, удаление исходных колонок (instruction, input, output, text).
3. Фильтрация итоговых промтов по максимальной длине в токенах.
4. Перемешивание датасета (shuffle) с инициализацией random seed.

In [32]:
def preprocess_dataset(tokenizer: AutoTokenizer, max_length: int, seed, dataset: str):
    """
    Tokenizes dataset for fine-tuning

    :param tokenizer (AutoTokenizer): Model tokenizer
    :param max_length (int): Maximum number of tokens to emit from the tokenizer
    :param seed: Random seed for reproducibility
    :param dataset (str): Instruction dataset
    """

    # Add prompt to each sample
    print("Preprocessing dataset...")
    dataset = dataset.map(create_prompt_formats)

    # Apply preprocessing to each batch of the dataset & and remove fields
    _preprocessing_function = partial(preprocess_batch, max_length = max_length, tokenizer = tokenizer)
    dataset = dataset.map(
        _preprocessing_function,
        batched = True,
        remove_columns = ['address', 'name_ru', 'rating', 'rubrics', 'text','key_words'],
    )

    # Filter out samples that have "input_ids" exceeding "max_length"
    dataset = dataset.filter(lambda sample: len(sample["input_ids"]) < max_length)

    # Shuffle dataset
    dataset = dataset.shuffle(seed = seed)

    return dataset

In [33]:
# Random seed
seed = 11111

max_length = get_max_length(model)
preprocessed_dataset = preprocess_dataset(tokenizer, max_length, seed, dataset)

Found max lenth: 4096
Preprocessing dataset...


Map:   0%|          | 0/498912 [00:00<?, ? examples/s]

Map:   0%|          | 0/498912 [00:00<?, ? examples/s]

Filter:   0%|          | 0/498912 [00:00<?, ? examples/s]

Вот так выглядит теперь датасет, состоящий из токенов

In [34]:
print(preprocessed_dataset)

Dataset({
    features: ['prompt', 'input_ids', 'attention_mask'],
    num_rows: 498911
})


In [35]:
print(preprocessed_dataset[0])

{'prompt': 'Категории:\nмагазин автозапчастей и автотоваров, автосервис, автотехцентр\n\nРейтинг:\n5.0\n\nКлючевые слова:\nавто, александру, благодарность\n\nОтзыв:\nпрекрасное вежливое обслуживание, (меняла зимние колеса на летние) записали и приняли в тот же день. отдельная благодарность мастеруприемщику александру за подробную консультацию по авто', 'input_ids': [1, 6881, 271, 1655, 434, 286, 13, 19232, 1906, 413, 504, 835, 2426, 263, 1906, 320, 1066, 292, 259, 1906, 10873, 259, 1906, 16401, 2906, 13, 13, 4586, 310, 8462, 286, 13, 301, 260, 272, 13, 13, 726, 966, 3129, 452, 816, 286, 13, 14429, 259, 305, 4120, 21804, 277, 259, 13821, 13, 13, 4672, 9250, 286, 13, 2083, 3368, 450, 9109, 271, 18924, 259, 308, 8899, 264, 261, 4753, 737, 14041, 265, 261, 21032, 335, 12690, 273, 263, 3696, 262, 603, 331, 572, 260, 23419, 13821, 2988, 277, 996, 446, 3588, 277, 305, 4120, 21804, 277, 280, 275, 5589, 420, 668, 8172, 2206, 275, 1906], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

### Создание конфигурации PEFT

Подход PEFT позволяет тюнить небольшое количество дополнительных параметров модели, одновременно замораживая большинство параметров предварительно обученных LLM, значительно снижая затраты на вычисления и хранение. Это также помогает обеспечить переносимость: пользователи могут настраивать модели с помощью методов PEFT, чтобы получить LoRa-модули в размером в несколько МБ.


Воспользуемся библиотекой `peft` из Hugging Face.

Существует несколько методов PEFT. Мы будем использовать QLoRA, применяя класс `LoraConfig` из библиотеки  `peft`.

QLoRA квантует модель в 4 бита, затем замораживает веса основной модели и добавляет две матрицы обучаемых мараметров. Во время тюнигна, QLoRA пробрасывает градиент через замороженную часть общей модели.

Обновляются только веса LoRa-модуля.

In [36]:
def create_peft_config(r, lora_alpha, target_modules, lora_dropout, bias, task_type):
    """
    Creates Parameter-Efficient Fine-Tuning configuration for the model

    :param r: LoRA attention dimension
    :param lora_alpha: Alpha parameter for LoRA scaling
    :param modules: Names of the modules to apply LoRA to
    :param lora_dropout: Dropout Probability for LoRA layers
    :param bias: Specifies if the bias parameters should be trained
    """
    config = LoraConfig(
        r = r,
        lora_alpha = lora_alpha,
        target_modules = target_modules,
        lora_dropout = lora_dropout,
        bias = bias,
        task_type = task_type,
    )

    return config

### Поиск модулей для LoRA

Функция `find_all_linear_names` предназначен для поиска слоёв оригинальной сети, для которых будет применяться LoRa.


Функция получает названия слоёв через `model.named_modules()`.

In [37]:
def find_all_linear_names(model):
    """
    Find modules to apply LoRA to.

    :param model: PEFT model
    """

    cls = bnb.nn.Linear4bit
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, cls):
            names = name.split('.')
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])

    if 'lm_head' in lora_module_names:
        lora_module_names.remove('lm_head')
    print(f"LoRA module names: {list(lora_module_names)}")
    return list(lora_module_names)

### Подсчёт обучаемых параметров

Функция `print_trainable_parameters` предназначена для расчёта количества обучаемых параметров в `model.named_parameters()`.

In [38]:
def print_trainable_parameters(model, use_4bit = False):
    """
    Prints the number of trainable parameters in the model.

    :param model: PEFT model
    """

    trainable_params = 0
    all_param = 0

    for _, param in model.named_parameters():
        num_params = param.numel()
        if num_params == 0 and hasattr(param, "ds_numel"):
            num_params = param.ds_numel
        all_param += num_params
        if param.requires_grad:
            trainable_params += num_params

    if use_4bit:
        trainable_params /= 2

    print(
        f"All Parameters: {all_param:,d} || Trainable Parameters: {trainable_params:,d} || Trainable Parameters %: {100 * trainable_params / all_param}"
    )

### Fine-tuning предобученной модели

Функция `fine_tune` оборачивает описанные выше модули и запускает их:


1. Разрешить сохранение градиентов (gradient checkpointing) для уменьшения использования памяти во время тюнинга.
2. Использование функции `prepare_model_for_kbit_training` из PEFT для подготови модели к тюнингу.
3. Вызов `find_all_linear_names` для получения названий слоёв сети для применения LoRA .
4. Создание конфигурации LoRA через вызов функции `create_peft_config`.
5. Оборачивание базовой модели с Hugging Face model для тюнинга через PEFT путём вызова функции `get_peft_model`.
6. Печать обучаемых параметров.


Для обучения мы инициализируем объект `Trainer()` внутри функции `fine_tune`, который требует:


`per_device_train_batch_size`: ,размер батча на обучении.


`gradient_accumulation_steps`: количество шагов, для которых необходимо накопить градиенты перед выполнением обратного прохода.


`warmup_steps`: количество шагов линейного увеличения скорости обучения от 0 до `learning_rate`.


`max_steps`: количество шагов обучения.


`learning_rate`: начальный  learning rate для Adam.


`fp16`: использовать ли 16-bit (mixed) обучение вместо  32-bit.


`logging_steps`: количество шагов между двумя логированиями.


`output_dir`: папка для сохранения логов и модели.


`optim`: оптимизатор для обучения.

In [39]:
def fine_tune(model,
          tokenizer,
          dataset,
          lora_r,
          lora_alpha,
          lora_dropout,
          bias,
          task_type,
          per_device_train_batch_size,
          gradient_accumulation_steps,
          warmup_steps,
          max_steps,
          learning_rate,
          fp16,
          logging_steps,
          output_dir,
          optim):
    """
    Prepares and fine-tune the pre-trained model.

    :param model: Pre-trained Hugging Face model
    :param tokenizer: Model tokenizer
    :param dataset: Preprocessed training dataset
    """

    # Enable gradient checkpointing to reduce memory usage during fine-tuning
    model.gradient_checkpointing_enable()

    # Prepare the model for training
    model = prepare_model_for_kbit_training(model)

    # Get LoRA module names
    target_modules = find_all_linear_names(model)

    # Create PEFT configuration for these modules and wrap the model to PEFT
    peft_config = create_peft_config(lora_r, lora_alpha, target_modules, lora_dropout, bias, task_type)
    model = get_peft_model(model, peft_config)

    # Print information about the percentage of trainable parameters
    print_trainable_parameters(model)

    # Training parameters
    trainer = Trainer(
        model = model,
        train_dataset = dataset,
        args = TrainingArguments(
            per_device_train_batch_size = per_device_train_batch_size,
            gradient_accumulation_steps = gradient_accumulation_steps,
            warmup_steps = warmup_steps,
            max_steps = max_steps,
            learning_rate = learning_rate,
            fp16 = fp16,
            logging_steps = logging_steps,
            output_dir = output_dir,
            optim = optim,
            report_to="tensorboard",####/

        ),
        data_collator = DataCollatorForLanguageModeling(tokenizer, mlm = False)
    )

    model.config.use_cache = False

    do_train = True

    # Launch training and log metrics
    print("Training...")

    if do_train:
        train_result = trainer.train()
        metrics = train_result.metrics
        trainer.log_metrics("train", metrics)
        trainer.save_metrics("train", metrics)
        trainer.save_state()
        print(metrics)

    # Save model
    print("Saving last checkpoint of the model...")
    os.makedirs(output_dir, exist_ok = True)
    trainer.model.save_pretrained(output_dir)

    # Free memory for merging weights
    del model
    del trainer
    torch.cuda.empty_cache()

Используем парамерты QLoRa для обучения

In [40]:
################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 16

# Alpha parameter for LoRA scaling
lora_alpha = 64

# Dropout probability for LoRA layers
lora_dropout = 0.1

# Bias
bias = "none"

# Task type
task_type = "CAUSAL_LM"

In [41]:
################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Batch size per GPU for training
per_device_train_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 4

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-5

# Optimizer to use
optim = "paged_adamw_32bit"

# Number of training steps (overrides num_train_epochs)
max_steps = 200 #0
# Linear warmup steps from 0 to learning_rate
warmup_steps = 2

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = True

# Log every X updates steps
logging_steps = 1

In [42]:
fine_tune(model,
      tokenizer,
      preprocessed_dataset,
      lora_r,
      lora_alpha,
      lora_dropout,
      bias,
      task_type,
      per_device_train_batch_size,
      gradient_accumulation_steps,
      warmup_steps,
      max_steps,
      learning_rate,
      fp16,
      logging_steps,
      output_dir,
      optim)

LoRA module names: ['down_proj', 'v_proj', 'q_proj', 'k_proj', 'gate_proj', 'o_proj', 'up_proj']
All Parameters: 3,540,389,888 || Trainable Parameters: 39,976,960 || Trainable Parameters %: 1.1291682911958425
Training...


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended,

Step,Training Loss
1,13.363500
2,13.774200
3,14.313500
4,14.061700
5,13.469700
6,12.753500
7,13.200500
8,12.726800
9,12.648400
10,12.117200


***** train metrics *****
  epoch                    =     0.0064
  total_flos               = 23278347GF
  train_loss               =     9.6873
  train_runtime            = 0:09:23.30
  train_samples_per_second =      5.681
  train_steps_per_second   =      0.355
{'train_runtime': 563.3045, 'train_samples_per_second': 5.681, 'train_steps_per_second': 0.355, 'total_flos': 2.499493575848755e+16, 'train_loss': 9.687315547466278, 'epoch': 0.0064139567699313706}
Saving last checkpoint of the model...


In [91]:
device = 'cuda'
def generate_review(category, rating, keywords, max_length=100, temperature=0.5, top_k=50, top_p=0.95):
    prompt = f"Категория: {category}\nРейтинг: {rating}\nКлючевые слова: {keywords}\nОтзыв: "
    inputs = tokenizer.encode(prompt, return_tensors='pt').to(device)

    with torch.no_grad():
        outputs = model.generate(
            inputs,
            max_length=inputs.shape[1] + max_length,
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
            do_sample=True,
            num_return_sequences=1,
            pad_token_id=tokenizer.eos_token_id
        )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    # Извлечение только текста отзыва

    return generated_text

In [44]:
category = "ресторан"
rating = 5
keywords = "паста, италия"

generated_review = generate_review(category, rating, keywords, 150, 1, 100000, 1.5)
print("Сгенерированный отзыв:")
print(generated_review)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Сгенерированный отзыв:
 4 июля мы праздновали день рождения.()`, решив втроём отмоститься в  всегда пьют итальянскую пасту с макаронами.хотя про один стол толькоический сериал проеспастал , последний вариант.тот.колебавшегося жрать.ведёмся с завтрака с 5 утра,чтобы вечером сделать многого уже оното чувствовался под его обширном, нарядным столом.кстати этот стол психолог организации необдуманное ребенка?так и есть!?попаска полышек то будто врубились в пятую точку!😉.хлебна ваниль могла бы быть нормальной,отвесняющая указала что не выпеченное,а


In [45]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [68]:
category = "ресторан"
rating = 3
keywords = "азиатская, китай, вкусно"

generated_review = generate_review(category, rating, keywords, 50, 0.3, 100, 0.3)
print("Сгенерированный отзыв:")
print(generated_review)

/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Сгенерированный отзыв:
 еда в азиатской кухне, но не очень. Категории китай, азиатская. еда не очень вкусная, но не очень дорого. еда не очень быстрая. официанты не очень приветливые.


In [82]:
category = "ресторан"
rating = 5
keywords = "паста, италия"

generated_review = generate_review(category, rating, keywords, 15, 0.1, 50, 0.5)
print("Сгенерированный отзыв:")
print(generated_review)

Сгенерированный отзыв:
 еда очень вкусная, персонал приветливый, но официанты не очень 


In [81]:
category = "жилой комплекс"
rating = 5
keywords = "метро, дорога, детский сад, школа"

generated_review = generate_review(category, rating, keywords, 20, 0.5, 150, 1)
print("Сгенерированный отзыв:")
print(generated_review)

Сгенерированный отзыв:
 1) удобное расположение.0) рядом с метро, 2) рядом с детским садом


In [80]:
category = "жилой комплекс"
rating = 1
keywords = "метро, школа, далеко"

generated_review = generate_review(category, rating, keywords, 20, 0.5, 150, 1)
print("Сгенерированный отзыв:")
print(generated_review)

Сгенерированный отзыв:
 1.0. далеко от метро, школа, центр, далеко. вообще не уютная территория


In [79]:
category = "жилой комплекс"
rating = 5
keywords = "метро, школа, далеко, машина"

generated_review = generate_review(category, rating, keywords, 20, 0.5, 150, 1)
print("Сгенерированный отзыв:")
print(generated_review)

Сгенерированный отзыв:
 10 минут пешком до метро.0,5 км до школы.1,5 км до детского


In [86]:
category = "ресторан"
rating = 5
keywords = "паста, италия"

generated_review = generate_review(category, rating, keywords, 50, 0.9, 500, 0.8)
print("Сгенерированный отзыв:")
print(generated_review)

Сгенерированный отзыв:
 атмосферное место с отличными итальянскими блюдами. Категории блюд как раз по нашим вкусам. любезен и вежливый персонал. рекомендую. особенно хочется отметить прекрасный десерт мороженый сырик с


In [88]:
category = "ресторан"
rating = 2
keywords = "паста, италия"

generated_review = generate_review(category, rating, keywords, 100, 0.9, 500, 0.8)
print("Сгенерированный отзыв:")
print(generated_review)

Сгенерированный отзыв:
 блюда были вкусно, но я считаю, что итальянская паста в Италии лучше чем здесь.0,5 из 5.
место не совсем подходящее для детей. детей не кормят, не дают напитки. видимо, это


In [95]:
category = "МФЦ"
rating = 5
keywords = "документы, заявление"

generated_review = generate_review(category, rating, keywords, 100, 0.7, 10000, 0.8)
print("Сгенерированный отзыв:")
print(generated_review.split('Отзыв:')[-1])

Сгенерированный отзыв:
 1.0. 2019. 15. 09. 12. 00. в МФЦ не смогли получить свидетельство о браке. 1. заявление заверено нотариусом, но не в том виде, в котором оно написано в приложении. 2. в заявлении написано, что мы не смогли получить свидетельство о браке, а не что то в этом роде. 3. не смогли получить заявление о браке. 4


In [93]:
category = "МФЦ"
rating = 2
keywords = "документы, заявление"

generated_review = generate_review(category, rating, keywords, 100, 0.7, 500, 0.8)
print("Сгенерированный отзыв:")
print(generated_review.split('Отзыв:')[-1])


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:632: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Сгенерированный отзыв:
 1.0. в первый раз в МФЦ. непонятные правила, непонятная очередь, непонятная процедура. в очереди стоит 15 минут. после записи заявления в очереди, получаешь номер. я был в очереди, но не мог получить номер. потом мне сказали, что не могу получить номер, потому что я не напишу заявление, а уже иду в кабинет. я объяснил, что я уже напишу заявление, но не могу получить номер. мне сказали,
